In [1]:
# 1. Các thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

In [2]:
# 2. Đọc file dữ liệu chứng khoán AAPL và chia train test validate theo tỉ lệ 7:1:2
df = pd.read_csv("../../dataset/KEY.csv")
df = df.dropna() # Drop missing values
df = df.reset_index(drop=True) # Reset the index
X = df[['Open']]
df = df[['Close']]


# Split the data into training, testing, and validation sets
train_size = int(0.7 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

In [3]:
#Trực quan các tệp dữ liệu vừa chia
plt.figure(figsize=(16,9))
plt.plot(train_data, 'blue', label='Training Data')
plt.plot(test_data, 'green', label='Testing Data')
plt.plot(val_data, 'red', label='Validation Data')
plt.legend()

In [4]:
# 3. Quá trình Training
x_train = np.array(train_data.index).reshape(-1, 1)
y_train = np.array(train_data['Close'])

# Tìm mô hình SARIMAX tốt nhất
model = auto_arima(y_train, exogenous=X[:train_size], 
                   start_p=1, start_q=1, 
                   test='adf',
                   max_p=3, max_q=3, m=12,
                   start_P=0, seasonal=True, 
                   d=None, D=1, trace=True, 
                   error_action='ignore', suppress_warnings=True,
                   stepwise=True)

model.fit(y_train)

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=7.80 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=5208.015, Time=0.44 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=456.903, Time=4.63 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=3159.707, Time=4.87 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=5218.420, Time=0.19 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=1042.329, Time=0.95 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=193.551, Time=9.78 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=22.66 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=7.56 sec
 ARIMA(0,0,0)(2,1,0)[12] intercept   : AIC=5189.819, Time=3.67 sec
 ARIMA(2,0,0)(2,1,0)[12] intercept   : AIC=195.366, Time=11.86 sec
 ARIMA(1,0,1)(2,1,0)[12] intercept   : AIC=195.382, Time=10.79 sec
 ARIMA(0,0,1)(2,1,0)[12] intercept   : AIC=3147.394, Time=6.62 sec
 ARIMA(2,0,1)(2,1,0)[12] intercept   : AIC=196.744, Time=13.20 sec
 ARIMA(1,0,0)(2,1,0)[12]           

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(2, 1, 0, 12),
      suppress_warnings=True, with_intercept=False)

In [5]:
# 4. Quá trình testing 
x_test = np.array(test_data.index).reshape(-1, 1)
y_test = np.array(test_data['Close'])
y_pred = model.predict(n_periods=len(y_test))


In [6]:
# 5. Quá trình Validate
x_val= np.array(val_data.index).reshape(-1, 1)
y_val = np.array(val_data['Close'])
y_pred_val =  model.predict(n_periods=len(y_val))


In [7]:
# 6. Quá trình tạo index predict 30 ngày tiếp theo
last_index =  df.index[-1]
last_data = pd.RangeIndex(start=last_index, stop=last_index+30, step=1)

# Create an array of 30 consecutive integers starting from last_index
x_next_30_days = np.array(range(last_index+1, last_index+31)).reshape(-1, 1)

# Predict the closing prices for the next 30 days
y_next_30_days = model.predict(n_periods=len(x_next_30_days))

# Print the predicted closing prices for the next 30 days
print('Predicted closing prices for the next 30 days:')
print(y_next_30_days)


Predicted closing prices for the next 30 days:
[16.52922842 16.53139094 16.66080015 16.69933875 16.41757191 16.52836353
 16.53403621 16.63768848 16.72549483 16.78701979 16.96586683 16.93941916
 16.65629171 16.67984208 16.83194783 16.84072098 16.51292712 16.73026088
 16.7143951  16.82373122 16.78988942 16.94138803 17.07314463 16.98229043
 16.72079426 16.67915505 16.81548507 16.83873998 16.38369531 16.62977202]


In [8]:
# 7. Đánh giá độ chính xác validate, test
valid_rmse = np.sqrt(np.mean((y_pred_val - y_val)**2))
valid_mae = np.mean(np.abs(y_pred_val - y_val))
valid_mape = np.mean(np.abs((y_pred_val - y_val) / y_val)) * 100

test_rmse = np.sqrt(np.mean((y_pred - y_test)**2))
test_mae = np.mean(np.abs(y_pred - y_test))
test_mape = np.mean(np.abs((y_pred - y_test) / y_test)) * 100

print('Validation RMSE:', valid_rmse)
print('Validation MAE:', valid_mae)
print('Validation MAPE:', valid_mape)

print('Testing RMSE:', test_rmse)
print('Testing MAE:', test_mae)
print('Testing MAPE:', test_mape)

Validation RMSE: 4.139120523831936
Validation MAE: 3.2119572732084993
Validation MAPE: 19.394179012369285
Testing RMSE: 4.21435612064746
Testing MAE: 3.594562438284381
Testing MAPE: 21.709760273532368


In [9]:
# 9.Vẽ hình
plt.figure(figsize=(16,9))
plt.plot(train_data.index, train_data['Close'])
plt.plot(test_data.index, test_data['Close'])
plt.plot(val_data.index, val_data['Close'])
plt.plot(test_data.index, y_pred)
plt.plot(val_data.index, y_pred_val)
plt.plot(last_data,y_next_30_days)
plt.legend(['Train', 'Test', 'Validate', 'Predictions','Valid_pred','Next30Day'])
plt.show()

In [10]:
test_data.index

RangeIndex(start=2362, stop=3037, step=1)